# MeshCat Animations

MeshCat.jl also provides an animation interface, built on top of the [three.js animation system](https://threejs.org/docs/#manual/introduction/Animation-system). While it is possible to construct animation clips and tracks manually, just as you would in Three.js, it's generally easier to use the MeshCat `Animation` type.

Let's show off building a simple animation. We first have to create our scene: 

In [ ]:
# Optional: 
# These commands tell the Julia package manager to use the exact
# set of dependencies specified in the Project.toml and
# Manifest.toml files in this folder. That should
# give you a nice, reproducible environment for testing. 

using Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

In [ ]:
using MeshCat, GeometryTypes, CoordinateTransformations

In [ ]:
vis = Visualizer()

In [ ]:
## To open the visualizer in a new browser tab, do: 
# open(vis)

## To open the visualizer inside this jupyter notebook, do: 
# IJuliaCell(vis)

## To open this visualizer in a standalone window, do:
# using Blink
# AtomShell.isinstalled() || AtomShell.install()
# open(vis, Window())

In [ ]:
setobject!(vis[:box1], 
    HyperRectangle(Vec(0., 0, 0), Vec(0.1, 0.2, 0.3)))

### Building an Animation

We construct an animation by first creating a blank `Animation()` object. We can then use the `atframe` function to set properties or transforms of the animation at specific frames of the animation. Three.js will automatically interpolate between whatever values we provide. 

For example, let's animate moving the box from [0, 0, 0] to [0, 1, 0]: 

In [ ]:
anim = Animation()

atframe(anim, vis, 0) do frame
    # `frame` behaves like a Visualizer, in that we can
    # call `settransform!` and `setprop!` on it, but
    # it just stores information inside the animation
    # rather than changing the current visualization
    settransform!(frame[:box1], Translation(0., 0, 0))
end

atframe(anim, vis, 30) do frame
    settransform!(frame[:box1], Translation(0., 1, 0))
end

# `setanimation!()` actually sends the animation to the
# viewer. By default, the viewer will play the animation
# right away. To avoid that, you can also pass `play=false`. 
setanimation!(vis, anim)

You should see the box slide 1 meter to the right in the viewer. If you missed the animation, you can run it again from the viewer. Click "Open Controls", find the "Animations" section, and click "play". 

### Animating the Camera

The camera is just another object in the MeshCat scene. To set its transform, we just need to index into the visualizer with the right path (note the leading `/`):

In [ ]:
settransform!(vis["/Cameras/default"], Translation(0, 0, 1))

To animate the camera, we just have to do that same kind of `settransform!` to individual frames in an animation: 

In [ ]:
anim = Animation()

atframe(anim, vis, 0) do frame
    settransform!(frame["/Cameras/default"], Translation(0., 0, 0))
end

atframe(anim, vis, 30) do frame
    settransform!(frame["/Cameras/default"], Translation(0., 0, 1))
end
 
setanimation!(vis, anim)

We can also animate object properties. For example, let's animate the camera's `zoom` property to smoothly zoom out and then back in. Note that to do this, we have to access a deeper path in the visualizer to get to the actual camera object. For more information, see: https://github.com/rdeits/meshcat#camera-control

In [ ]:
anim = Animation()

atframe(anim, vis, 0) do frame
    setprop!(frame["/Cameras/default/rotated/<object>"], "zoom", 1)
end

atframe(anim, vis, 30) do frame
    setprop!(frame["/Cameras/default/rotated/<object>"], "zoom", 0.5)
end

atframe(anim, vis, 60) do frame
    setprop!(frame["/Cameras/default/rotated/<object>"], "zoom", 1)
end
 
setanimation!(vis, anim)

### Recording an Animation

To record an animation at a smooth, fixed frame rate, click on "Open Controls" in the viewer, and then go to "Animations" -> "default" -> "Recording" -> "record". This will play the entire animation, recording every frame and then let you download the resulting frames to your computer. 

To record activity in the MeshCat window that isn't a MeshCat animation, we suggest using a screen-capture tool like Quicktime for macOS or RecordMyDesktop for Linux. 

### Converting the Animation into a Video

Currently, meshcat can only save an animation as a `.tar` file consisting of a  list of `.png` images, one for each frame. To convert that into a video, you will need to install the `ffmpeg` program, and then you can run: 

In [ ]:
# MeshCat.convert_frames_to_video("/home/rdeits/Downloads/meshcat_1528401494656.tar", overwrite=true)